# Flask и немного магии

## Словари в шаблонах

Предположим, у нас есть словарь, в котором хранятся имена наших друзей и их почтовые адреса. И мы хотим выводить эти имена с адресами на нашем сайте. Мы можем написать что-то такое:

In [4]:
from flask import Flask
from flask import render_template

app = Flask(__name__)


@app.route('/')
def index():
    emailbook = {'Петя': 'petya@example.com',
                 'Вася': 'vasya@example.com',
                 'Катя': 'katya@example.com'}
    return render_template('index.html', emails=emailbook)

if __name__ == '__main__':
    app.run()

Тогда в папке `templates` нам нужно создать файл `index.html`, в котором будут перебираться элементы словаря. Делается это с помощью функции `items()`. Вот так будет выглядеть `index.html`:

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Почтовые адреса</title>
</head>
<body>
<h1>Адреса моих друзей</h1>
<ul>
    {% for name, email in emails.items() %}
        <li>{{ name }} - {{ email }}</li>
    {% endfor %}
</ul>
</body>
</html>

## Перенаправление

Для того, чтобы направить пользователя на другую страницу, используется функция `redirect`. Например, вот приложение, в котором есть страница /time. С помощью функции `redirect` можно, например, реализовать такое:

- если пользователь заходит на страницу /time в рабочее время (с 10 до 18), то он перенаправляется на главную страницу сайта,
- если пользователь заходит на страницу в другое время, то он перенаправляется на страницу /hi.

In [ ]:
import datetime

from flask import Flask
from flask import url_for, render_template, request, redirect

app = Flask(__name__)


@app.route('/')
def index():
    return '<html><body><h1>Привет, мир!</h1></body></html>'


@app.route('/hi')
@app.route('/hi/<user>')
def hi(user=None):
    if user is None:
        user = 'friend'
    return '<html><body><p>Привет, ' + user + '!</p></body></html>'


@app.route('/time')
def time_redirect():
    h = datetime.datetime.today().hour
    if 10 < h < 18:
        return redirect(url_for('index'))
    return redirect(url_for('hi'))


if __name__ == '__main__':
    app.run(debug=True)

Когда вы пишете сайт на фласке у себя на компьютере, можно написать вместо `app.run()` вот так: `app.run(debug=True)` (см. выше последнюю строчку). Это значит, что если на сайте возникнет ошибка, то на странице с ошибкой выведется ее описание, и вы легко сможете понять, в каком месте кода все падает. Но когда вы будете выкладывать сайт в Интернет, нужно убедиться, что `debug` отключен.

## Как вывести на веб-страничку содержимое текстового файла?

In [10]:
from flask import Flask
from flask import url_for, render_template, request, redirect

app = Flask(__name__)

@app.route('/poem')
def poem():
    with open("poem.txt", "r", encoding='utf-8') as f:
        content = f.read().split('\n')
    return render_template("poem.html", content=content)

if __name__ == '__main__':
    app.run(debug=True)

In [8]:
<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
    <title>Зимнее утро</title>
</head>
<body>
    <p>
    {%for i in content%}
        {{i}}
        <br>
    {%endfor%}
    </p>
</body>
</html>

## Чего делать не надо?

1. Самое главное - не называть рабочий файл так же, как модуль. Если вы назовете файл flask.py, то получите ошибку импорта имени Flask, потому что ваш файл будет импортировать сам себя.
2. Кроме того, не надо называть свои переменные и функции так же, как называются встроенные функции в питоне, т.е. переменные с именами dict и list - плохая идея.

## Как проверить версию питона и список установленных пакетов?

* Чтобы посмотреть версию питона, нужно открыть терминал (на Windows он называется cmd, можно найти через поиск) и написать там `python --version`. В ответ вы должны получить что-то вроде `Python 3.6.0 :: Anaconda custom (64-bit)`.
* Стандартный терминал на Windows очень неудобный. Альтернативу можно выбрать, прочитав [вот эту статью](https://habrahabr.ru/post/164687/).
* Чтобы увидеть список установленных пакетов, нужно набрать в терминале `pip freeze`. На выходе вы увидите что-то такое:

`
regex==2017.9.23
requests==2.14.2
rope-py3k==0.9.4.post1
scikit-image==0.13.0
scikit-learn==0.19.0
scipy==0.19.1
seaborn==0.7.1
simplegeneric==0.8.1
singledispatch==3.4.0.3
six==1.10.0
snowballstemmer==1.2.1
sockjs-tornado==1.0.3
spacy==1.9.0
`

* Если у вас старая версия питона, то вы можете увидеть ошибку, что pip не найден. Тогда его нужно установить вручную.  Это делается командой `python get-pip.py`. Подробнее о работе с pip можно почитать на [официальном сайте с документацией](https://pip.pypa.io/en/stable/)
* pip позволяет устанавливать новые пакеты простой командой `pip install package-name`. Если вы устанавливали сборку Anaconda, то новые пакеты также можно устанавливать командой `conda install package-name`.
* На компьютере может быть установлено больше одного интерпретатора питона, например, python2 и python3. Соответственно, на разные интерпретаторы могут быть установлены разные пакеты. Интерпретатор - это программа, которая делает код, написанный на высокоуровневом языке вроде питона, понятным компьютеру, который оперирует низкоуровневыми командами. В нашем случае интерпретатор - это файл python, который исполняет все скрипты с расширением `.py`. По умолчанию скрипты исполняются тем интерпретатором, который выдаст вам команда `python --version` в терминале.
* Если при вводе `python` в командную строку появляется ошибка, что такой команды нет, или же вы хотите сделать дефолтной другую версию питона, нужно прописать путь к питону в переменной среды (окружения) PATH. Эта переменная содержит список **директорий**, в которых операционная система должна искать исполняемые файлы. В качестве разделителя используется точка с запятой (;) в операционной системе Windows и двоеточие (:) в операционных системах Linux и MacOS. Очень хорошее краткое объяснение переменных среды в целом и PATH в частности можно почитать [вот тут](http://barancev.github.io/what-is-path-env-var/). На Windows добавить новую строку в PATH можно через `Изменение системных переменных среды > Переменные среды > PATH > Изменить`. В разных версиях и локализациях Windows это может называться "переменными окружения" или "environment variables". Например, если у вас анаконда, то в папке должны быть строки следующего типа:

`C:\Users\ancatmara\Anaconda3
C:\Users\ancatmara\Anaconda3\Lib\bin
C:\Users\ancatmara\Anaconda3\Scripts`

А если обычный питон - то что-то вот такое.

`C:\ProgramFiles\Python3
C:\ProgramFiles\Python3\Lib
C:\ProgramFiles\Python3\Scripts`

* В различных IDE (IDLE, PyCharm и т.д.) тоже можно выбирать интерпретатор в настройках. Например, в PyCharm это делается с помощью `File > Settings > Project > Project Interpreter`. На всякий случай напоминаю, что IDE значит *Integrated Development Environment*, т.е. интегрированная среда разработки. Это такая специальная программа, которая делает написание кода на определенном языке гораздо удобнее, своего рода редактор.

## Задание для тренировки

Скачайте из репозитория [список языковых кодов](https://github.com/elmiram/2016learnpython/blob/master/data/lang_codes.csv). 
После этого нужно создать сайт, в котором:
1. На главной странице выводится список языков и их кодов (`127.0.0.1:5000`),
2. Если зайти на страницу вида `127.0.0.1:5000/langs/какие-то_буквы`, то на этой странице выведется список только тех языков, коды которых начинаются с этих букв. Например, если зайти на `127.0.0.1:5000/codes/jp`, то выведется японский и еврейско-персидский, а если зайти на `127.0.0.1:5000/codes/j`, то японский, еврейско-персидский, еврейско-арабский и яванский.
3. Есть форма на странице `127.0.0.1:5000/form`, в которой есть поле "Язык". В это поле пользователь будет вводить название языка, например "японский". Когда пользователь нажимает кнопку "Отправить", 
    * если введенного языка нет в списке, пользователь попадает на страницу `127.0.0.1:5000/not_found` с соответствующим сообщением,
    * если введенный язык есть в списке, пользователь должен попасть на страницу `127.0.0.1:5000/lang/код_языка`. На этой странице должно выводиться следующее: название введенного языка, код введенного языка, названия всех языков, код которых начинается на ту же букву.
4. На всех страницах есть кликабельная ссылка на главную и на страницу с формой.

Пример сайта на фласке есть в репозитории [вот тут](https://github.com/ancatmara/learnpython2017/tree/master/%D0%A1%D0%B5%D0%BC%D0%B8%D0%BD%D0%B0%D1%80%D1%8B/flask_example). Там есть примеры использования всех функций фласка, которые мы проходили. (*Туда можно подглядывать.*)